https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 18.49 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.6 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-11-18 07:59:48--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response

In [ ]:
import pandas as pd
import numpy as np
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
from collections import Counter # 단어 등장 횟수 카운트

In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f55004ba00f896950f6de8fa85f6a3cb53e932f3076eaa59950d666e8f46be56
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipyt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 31.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
dataset = pd.read_csv('/content/Market_kurly.csv')
dataset

,title,nickname,datetime,content
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202..."
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...
...,...,...,...,...
3988,마켓컬리 컬리카드 장바구니혜택 챙겨요!,뷰스타 카민,2023. 5. 4. 12:00,리뷰\n마켓컬리 컬리카드 장바구니혜택 챙겨요!\n뷰스타 카민 ・ 2023. 5. 4...
3989,마켓컬리 추천상품 툭툭누들타이 사왓디만두,우리의 하루,2023. 1. 28. 9:00,레시피\n마켓컬리 추천상품 툭툭누들타이 사왓디만두\n우리의 하루 ・ 2023. 1....
3990,마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천,호냥,2023. 6. 5. 14:10,일상스토리\n마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천\n호냥 ・ 2023. 6....
3991,"쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 11번가 슈팅배...",각인,2023. 7. 21. 17:12,"정보/감상\n쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 1..."


In [ ]:
# txt 파일에서 stopword를 읽어오기
with open('/content/stopwords.txt', 'r', encoding='UTF-8') as file:
    stopword_data = file.read()


# 쉼표로 구분된 데이터를 리스트로 변환
stopword_list = stopword_data.split(',')
stopword_list = pd.DataFrame(stopword_list)
stopword_list.head()

,0
0,휴
1,아이구
2,아이쿠
3,아이고
4,어


In [ ]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['content']))
dataset['content_removed'] = review_removed
dataset

,title,nickname,datetime,content,content_removed
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...,리뷰마켓컬리 주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게고...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...,식품 리뷰마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...,블챌 체크인 챌린지자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천영앤지...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202...",내돈내산직사용후기낭만어시장 땡초넣은 모듬 조개탕 마켓컬리단감 복사 이웃추가...
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...,엄마의 장바구니마켓컬리 첫구매 신규가입 쿠폰 추천상품행복바이러스 복사 이웃...
...,...,...,...,...,...
3988,마켓컬리 컬리카드 장바구니혜택 챙겨요!,뷰스타 카민,2023. 5. 4. 12:00,리뷰\n마켓컬리 컬리카드 장바구니혜택 챙겨요!\n뷰스타 카민 ・ 2023. 5. 4...,리뷰마켓컬리 컬리카드 장바구니혜택 챙겨요뷰스타 카민 복사 이웃추가본문 기타...
3989,마켓컬리 추천상품 툭툭누들타이 사왓디만두,우리의 하루,2023. 1. 28. 9:00,레시피\n마켓컬리 추천상품 툭툭누들타이 사왓디만두\n우리의 하루 ・ 2023. 1....,레시피마켓컬리 추천상품 툭툭누들타이 사왓디만두우리의 하루 복사 이웃추가본문...
3990,마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천,호냥,2023. 6. 5. 14:10,일상스토리\n마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천\n호냥 ・ 2023. 6....,일상스토리마켓컬리 오뗄블랙라벨 전지 베이컨베이컨추천호냥 복사 이웃추가본문 ...
3991,"쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 11번가 슈팅배...",각인,2023. 7. 21. 17:12,"정보/감상\n쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 1...",정보감상쇼핑을 할땐 신규첫구매 쿠폰을 활용하세요 마켓컬리 번가 슈팅배송 오늘의집각...


In [ ]:
review_tokenized = dataset['content_removed'].apply(lambda review: okt.nouns(review))

In [ ]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list[0]):
                review_removed_stopword.append(token)

    return review_removed_stopword

In [ ]:
dataset['content_removed_stopword'] = review_tokenized.apply(remove_stopword).replace(",", "")
content_removed_stopword = dataset['content_removed_stopword']

In [ ]:
dataset['content_removed_stopword'] = content_removed_stopword
dataset

,title,nickname,datetime,content,content_removed,content_removed_stopword
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...,리뷰마켓컬리 주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게고...,"[역대, 상품, 포도, 계란, 소고기, 역대, 상품, 포도, 계란, 소고기, 네이버..."
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...,식품 리뷰마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보...,"[식품, 떡볶이, 분식집, 떡볶이, 추억, 국민학교, 떡볶이, 요코, 능지, 떡볶이..."
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...,블챌 체크인 챌린지자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천영앤지...,"[자취, 헤비, 키트, 앤지, 위치, 역삼, 자취, 키트, 욥저, 재주, 편이, 자..."
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202...",내돈내산직사용후기낭만어시장 땡초넣은 모듬 조개탕 마켓컬리단감 복사 이웃추가...,"[산직, 낭만, 어시장, 땡초, 모듬, 조개, 단감, 사랑, 드릴, 낭만, 어시장,..."
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...,엄마의 장바구니마켓컬리 첫구매 신규가입 쿠폰 추천상품행복바이러스 복사 이웃...,"[신규가입, 쿠폰, 상품, 행복, 바이러스, 신규가입, 인기, 상품, 혜택, 쿠폰,..."
...,...,...,...,...,...,...
3988,마켓컬리 컬리카드 장바구니혜택 챙겨요!,뷰스타 카민,2023. 5. 4. 12:00,리뷰\n마켓컬리 컬리카드 장바구니혜택 챙겨요!\n뷰스타 카민 ・ 2023. 5. 4...,리뷰마켓컬리 컬리카드 장바구니혜택 챙겨요뷰스타 카민 복사 이웃추가본문 기타...,"[카드, 혜택, 카민, 평상시, 정보, 배송, 제품, 자꾸만, 자취, 면밀, 키트,..."
3989,마켓컬리 추천상품 툭툭누들타이 사왓디만두,우리의 하루,2023. 1. 28. 9:00,레시피\n마켓컬리 추천상품 툭툭누들타이 사왓디만두\n우리의 하루 ・ 2023. 1....,레시피마켓컬리 추천상품 툭툭누들타이 사왓디만두우리의 하루 복사 이웃추가본문...,"[레시피, 상품, 툭툭, 타이, 왓디, 안녕하세요우, 서비스, 샛별, 배송, 배송,..."
3990,마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천,호냥,2023. 6. 5. 14:10,일상스토리\n마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천\n호냥 ・ 2023. 6....,일상스토리마켓컬리 오뗄블랙라벨 전지 베이컨베이컨추천호냥 복사 이웃추가본문 ...,"[스토리, 블랙, 라벨, 전지, 베이컨, 베이컨, 쇼핑, 블랙, 라벨, 시리즈, 전..."
3991,"쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 11번가 슈팅배...",각인,2023. 7. 21. 17:12,"정보/감상\n쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 1...",정보감상쇼핑을 할땐 신규첫구매 쿠폰을 활용하세요 마켓컬리 번가 슈팅배송 오늘의집각...,"[정보, 감상, 쇼핑, 쿠폰, 활용, 번가, 슈팅, 배송, 각인, 네이버, 쿠팡, ..."


In [ ]:
# 각 내부 리스트의 모든 요소를 하나의 문자열로 합친 후 리스트 생성
output_list = [' '.join(item) for item in content_removed_stopword]

# 데이터프레임에 새로운 칼럼 추가
dataset['new_column'] = output_list
new_column = dataset['new_column']

In [ ]:
documents = list()
documents = new_column.values.tolist()
print(documents[:5])

['역대 상품 포도 계란 소고기 역대 상품 포도 계란 소고기 네이버 플루 애정 벌써 역대 상품 카드 정보 이벤트 진행 이벤트 월요일 반값 상품 상품 나볼 일일 가도 기회 식탁 상품 쟁여둘 수록 이득 쟁여템 퀄리티 쿠폰 원가 입시 쿠폰 쿠폰 가기 장점 자재 상품 배송 채소 먹거리 아침 주부 직장인 상품 퀄리티 역대 규모 상품 제품 반값 이벤트 반값 날씨 채소 과일 칠레 포도 반값 과일 가족 반값 스틱 자외선 차단 스틱 여름 대비 기회 일일 가일 월요일 상품 음식 가득 기회 인기 제품 이벤트 제품 정해진 개수 인기 국탕류 인기 반찬 평상시 음식 놨으 전단 음식 종류 평상시 필수품 계란 화장지 아기 팬티 기저귀 율도 기회 스윗 아이스크림 단것 다이어터 칼로리 아이스크림 여름 페스 한정 반값 혜택 플렉스 쿠폰 플렉스 쿠폰 쿠폰 쿠폰 플렉스 상품 주방 프리미엄 식품 한정 롯데 현대 신한 하나카드 카드 제휴 이벤트 마눤 카드 쿠폰 적립 혜택 상품 이벤트 역대 보지 혜택 윤택 생활 바로가기 소비자 커머스 리세', '식품 떡볶이 분식집 떡볶이 추억 국민학교 떡볶이 요코 능지 떡볶이 추억 국민학교 떡볶이 추억 국민학교 떡볶이 냉동식품 조리법 어묵 기름 포장 플라스틱 비닐 플라스틱 케이스 추억 국떡 밀떡 빨강 소스 소스 튀김 어묵 찰떡 찬물 요한 냄비 빨강 소스 소스 소스 종이컵 가득 어묵 찰떡 소스 식용유 스푼 어묵 대파 양파 달걀 라면 소스 쫄면 떡볶이 국떡 떡볶이 요밀 찰떡 쫄깃해 두께 양념 추억 분식집 떡볶이 국물 라면 유부초밥 김말이 튀김 떡볶이 소스 주말 술안주 최고 떡볶이 소스 떡볶이 떡볶이 추억 국민학교 떡볶이 냉동식품 추억 분식집 떡볶이', '자취 헤비 키트 앤지 위치 역삼 자취 키트 욥저 재주 편이 자취 식사 해결 키트 서울 맛집 레시피 대체로 평타 이상은 중앙 감속 우정 냉동식품 성수 최현석 셰프 중앙 감속 중앙 감속 구성 중앙 감속 튀김 소스 소스 가니쉬 구성 구가 자취 소스 가니쉬 조리 도로 분분 조리 에프 후라이팬 기름 소스 발사믹 베이스 어간 가니쉬 

In [ ]:
len(documents)

3993

In [ ]:
vectorizer = CountVectorizer(max_features=200)

sklearn.feature_extraction.text.CountVectorizer

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(3993, 900)


In [ ]:
vocab = vectorizer.get_feature_names_out()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [ ]:
len(vocab)

900

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=70)
ctm.fit(training_dataset)

Epoch: [70/70]	 Seen Samples: [279510/279510]	Train Loss: 462.47604083513335	Time: 0:00:02.185311: : 70it [03:15,  2.79s/it]


features=2000 Epoch =50	n_components=10 Train Loss: 694.969898983377
features=1000 Epoch =50	n_components=50 Train Loss: 484.9931330022853
features=1000 Epoch =50	n_components=60 Train Loss: 493.91621763868017
features=1000 Epoch =60	n_components=50 Train Loss: 484.9005804102492
features=1000 Epoch =70	n_components=50 Train Loss: 481.3086715228212
features=900 Epoch =70	n_components=50 Train Loss: 462.47604083513335

In [ ]:
ctm.get_topics()

defaultdict(list,
            {0: ['서비스',
              '시장',
              '네이버',
              '상장',
              '제공',
              '진행',
              '기업',
              '경험',
              '마케팅',
              '상황'],
             1: ['핫도그',
              '돈까스',
              '샌드위치',
              '프라이어',
              '치킨',
              '등심',
              '냉동',
              '소세지',
              '소시지',
              '떡갈비'],
             2: ['파운드',
              '간편식',
              '크로와상',
              '냉동',
              '제품',
              '디저트',
              '프랑스',
              '조리',
              '냉동식품',
              '초코'],
             3: ['딤섬',
              '제이드',
              '크리스탈',
              '트러플',
              '만두',
              '조리',
              '새우',
              '전자레인지',
              '프리미엄',
              '방법'],
             4: ['페스',
              '부스',
              '시식',
              '아트',
              '티켓',
              '하겐다즈',
            

In [ ]:
ctm.get_topic_lists(5)

[['서비스', '시장', '네이버', '상장', '제공'],
 ['핫도그', '돈까스', '샌드위치', '프라이어', '치킨'],
 ['파운드', '간편식', '크로와상', '냉동', '제품'],
 ['딤섬', '제이드', '크리스탈', '트러플', '만두'],
 ['페스', '부스', '시식', '아트', '티켓'],
 ['버터', '치즈', '시나몬', '크림', '피칸'],
 ['혜택', '카드', '상품', '적립금', '프로모션'],
 ['치즈', '파스타', '국산', '기획전', '소스'],
 ['상품', '우유', '제품', '배송', '적립금'],
 ['쿠폰', '적립금', '신규가입', '식비', '혜택'],
 ['떡볶이', '닭갈비', '국물', '춘천', '소스'],
 ['쿠폰', '배송', '적립금', '상품', '신규가입'],
 ['국물', '짬뽕', '목란', '일드', '이연'],
 ['짬뽕', '짜장면', '목란', '뇨끼', '파스타'],
 ['메밀', '육수', '막국수', '소바', '곰탕'],
 ['이벤트', '적립금', '신규가입', '행사', '더블'],
 ['비빔국수', '순두부', '망향', '키트', '마제'],
 ['김치', '상품', '제품', '배송', '우동'],
 ['조리', '프라이어', '막창', '감자', '돈까스'],
 ['민민', '여기저기', '유부초밥', '전복', '미역국'],
 ['키트', '갈비탕', '쌀국수', '냉면', '전골'],
 ['커피', '원두', '과일', '수박', '라떼'],
 ['부대찌개', '해장국', '닭갈비', '통뼈', '밀면'],
 ['배송', '퍼플', '포장', '샛별', '필터'],
 ['식빵', '감자칩', '버터', '막걸리', '잠봉'],
 ['우유', '도넛', '케이크', '딸기', '복음'],
 ['아보카도', '양파', '화분', '방울토마토', '게임'],
 ['피부', '제품', '로제', '헤어', '화장품'],
 ['요거트', '그릭

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)